Overload Signatures Examples

In [3]:
import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...
def sum(it, /, start=0):
    return functools.reduce(operator.add, it, start)


Max Overload Examples

In [4]:
from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
LT = TypeVar('LT', bound=SupportsLessThan)
DT = TypeVar('DT')

MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'

# These SupportLessThan, but have no provided key or default.
@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT:
    ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:
    ...
# These provide and argument but have no default.
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T:
    ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T:
    ...
# These provide a default but no key.
@overload
def max(__iterable: Iterable[LT], *, key: None = ..., default: DT) -> Union[LT, DT]:
    ...
# The provide a key and default.
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT], default: DT) -> Union[T, DT]:
    ...

TypedDict Examples

In [5]:
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

pp = BookDict(title='Programming', authors='Jon Bentley', isbn='0123123', pagecount=256)
print(pp)
print(type(pp))
# print(pp.title) # raises AttributeError

{'title': 'Programming', 'authors': 'Jon Bentley', 'isbn': '0123123', 'pagecount': 256}
<class 'dict'>


In [6]:
from typing import TYPE_CHECKING

def demo() -> None:
    book = BookDict(
        isbn='231432',
        title='Refactoring',
        authors=['AuthorA','AuthorB'],
        pagecount=1242
    )
    authors = book['authors']
    if TYPE_CHECKING:
        reveal_type(authors)
    authors = 'Bob'
    book['weight'] = 4.2
    del book['title']



In [7]:
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'

def to_xml(book: BookDict) -> str:
    elements: list[str] = []
    for key, value in book.items():
        if isinstance(value, list):
            elements.extend(
                AUTHOR_ELEMENT.format(n) for n in value)
        else:
            tag = key.upper()
            elements.append(f'<{tag}>{value}</{tag}>')
    xml = '\n\t'.join(elements)
    return f'<BOOK>\n\t{xml}\n</BOOK>'

In [8]:
import json
# def from_json(data: str) -> BookDict:
#     whatever = json.loads(data)
#     return whatever

def from_json(data: str) -> BookDict:
    whatever: BookDict = json.loads(data)
    return whatever

In [9]:
from typing import TYPE_CHECKING

def demo() -> None:
    NOT_BOOK_JSON = """
        {"title": "A_Title",
        "flavor": "pistachio",
        "authors": true}
    """
    not_book = from_json(NOT_BOOK_JSON)
    if TYPE_CHECKING:
        reveal_type(not_book)
        reveal_type(not_book['authors'])
    
    print(not_book)
    print(not_book['flavor'])
    
    xml = to_xml(not_book)
    print(xml)

demo()

{'title': 'A_Title', 'flavor': 'pistachio', 'authors': True}
pistachio
<BOOK>
	<TITLE>A_Title</TITLE>
	<FLAVOR>pistachio</FLAVOR>
	<AUTHORS>True</AUTHORS>
</BOOK>


Type Casting Examples

In [10]:
def cast(typ, val):
    """Cast a value to a type"""
    return val

def find_first_str(a: list[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))
    # If there is atleast one string:
    return cast(str, a[index])

Implementing a Generic Class

In [15]:
import random
from tombola import Tombola

class LottoBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)


# machine = LottoBlower[int](range(1,11))

# first = machine.pick()
# remain = machine.inspect()


# machine = LottoBlower[int]([1, .2]) # Returns error because .2 is a float not an int.

# machine.load('ABC') # Returns error for the string being incompatible.

In [16]:
import random

from collections.abc import Iterable
from typing import TypeVar, Generic

from tombola import Tombola

T = TypeVar('T')

class LottoBlower(Tombola, Generic[T]):
    
    def __init__(self, items: Iterable[T]) -> None:
        self._balls = list[T](items)

    def load(self, items: Iterable[T]) -> None:
        self._balls.extend(items)

    def pick(self) -> T:
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self) -> bool:
        return bool(self._balls)

    def inspect(self) -> tuple[T, ...]:
        return tuple(self._balls)

Variance Examples

In [17]:
# Invariance Example
from typing import TypeVar, Generic

class Beverage:
    """Any beverage."""

class Juice:
    """Any fruit juice."""

class OrangeJuice:
    """Delicious juice from Brazilian oranges."""

T = TypeVar('T')

class BeverageDispenser(Generic[T]):
    """A dispenser parameterized on the beverage type."""
    def __init__(self, beverage: T):
        self.beverage = beverage
    
    def dispense(self) -> T:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Install a fruit juice dispenser."""

# Valid Code
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)

# Invalid Code
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
# install(orange_juice_dispenser) # Not a valid type to install

In [18]:
# Covariant Example

T_co = TypeVar('T_co', covariant=True)

class BeverageDispenser(Generic[T_co]):
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage
    
    def dispense(self) -> T_co:
        return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Install a fruit juice dispenser."""


# Valid Code
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)

# Valid Code
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser) # Now Valid


# Invalid Code
beverage_dispenser = BeverageDispenser(Beverage())
# install(beverage_dispenser) # Not a valid type to install

In [19]:
from typing import TypeVar, Generic

class Refuse:
    """Any refuse."""

class Biodegradable(Refuse):
    """Biodegradable refuse."""

class Compostable(Biodegradable):
    """Compostable refuse."""

T_contra = TypeVar('T_contra', contravariant=True)

class TrashCan(Generic[T_contra]):
    def put(self, refuse: T_contra) -> None:
        """Store trash until dumped."""
    
def deploy(trash_can: TrashCan[Biodegradable]):
    """Deploy a trash can for biodegradable refuse."""

# Valid Code
bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can)

# Valid Code
trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can)

# Invalid Code
compost_can: TrashCan[Compostable] = TrashCan()
# deploy(compost_can) # Compostable is not a valid type


Implementing a Generic Static Protocol Examples

In [20]:
from typing import runtime_checkable, abstractmethod, TypeVar

T_co = TypeVar('T_co', covariant=True)

@runtime_checkable
class SupportsAbs(Protocol[T_co]):
    __slots__ = ()
    
    @abstractmethod
    def __abs__(self) -> T_co:
        pass


In [23]:
import math
from typing import NamedTuple, SupportsAbs

class Vector2d(NamedTuple):
    x: float
    y: float

    def __abs__(self) -> float:
        return math.hypot(self.x, self.y)

def is_unit(v: SupportsAbs[float]) -> bool:
    """'True' if the magnitude of 'v' is close to 1."""
    return math.isclose(abs(v), 1.0)

assert issubclass(Vector2d, SupportsAbs)

v0 = Vector2d(0, 1)
sqrt2 = math.sqrt(2)
v1 = Vector2d(sqrt2 / 2, sqrt2 / 2)
v2 = Vector2d(1, 1)
v3 = complex(.5, math.sqrt(3) / 2)
v4 = 1

assert is_unit(v0)
assert is_unit(v1)
assert not is_unit(v2)
assert is_unit(v3)
assert is_unit(v4)
print('OK')

OK


In [ ]:
from typing import Protocol, runtime_checkable, TypeVar

T_co = TypeVar('T_co', covariant=True)

@runtime_checkable
class RandomPicker(Protocol[T_co]):
    def pick(self) -> T_co: ...